1 - importação das bibliotecas:

In [1]:
import pandas as pd 
import plotly.express as px
import os
import zipfile
from plotly.subplots import make_subplots
pd.options.display.max_columns = None
pd.options.display.max_rows = None

2 - download do dataset:

In [2]:
if not os.path.exists("content"):
    os.makedirs("content")

os.system('kaggle datasets download -d vivek468/superstore-dataset-final -p content')

zip_path = "content/superstore-dataset-final.zip"
if os.path.exists(zip_path):

    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall("content")
    print("Arquivo descompactado com sucesso!")
else:
    print("Arquivo ZIP não encontrado!")

Arquivo descompactado com sucesso!


In [2]:
df = pd.read_csv("content/Sample - Superstore.csv", encoding='latin1')

3 - analise dos dados

3.1 - analise dos 5 primeiros e dos 5 ultimos dados da tabela

In [3]:
df.head() #analise das 5 primeiras colunas

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,State,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
0,1,CA-2016-152156,11/8/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600,2,0.00,41.9136
1,2,CA-2016-152156,11/8/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400,3,0.00,219.5820
2,3,CA-2016-138688,6/12/2016,6/16/2016,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,California,90036,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200,2,0.00,6.8714
3,4,US-2015-108966,10/11/2015,10/18/2015,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775,5,0.45,-383.0310
4,5,US-2015-108966,10/11/2015,10/18/2015,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680,2,0.20,2.5164


In [ ]:
df.tail() #anaise das 5 ultimas colunas

3.2 - nome das colunas e numero das linhas:

In [4]:
print("numero de linhas",df.shape[0])
print("nome das colunas:")
for n , i  in enumerate(df.columns,1):
    print("  ",n,":",i)

numero de linhas 9994
nome das colunas:
   1 : Row ID
   2 : Order ID
   3 : Order Date
   4 : Ship Date
   5 : Ship Mode
   6 : Customer ID
   7 : Customer Name
   8 : Segment
   9 : Country
   10 : City
   11 : State
   12 : Postal Code
   13 : Region
   14 : Product ID
   15 : Category
   16 : Sub-Category
   17 : Product Name
   18 : Sales
   19 : Quantity
   20 : Discount
   21 : Profit


3.3 - descrição do dataset

In [ ]:
df.describe().round().T

In [ ]:
df.info()

3.4 - modificação do tipo de dados para tabela "order data" e "Ship Date" para Datetime

In [5]:
df['Order Date']=df['Order Date'].astype('datetime64[ns]')
df['Ship Date']=df['Ship Date'].astype('datetime64[ns]')

3.5 - analise de valores nulos:

In [6]:
df.isna().sum().sum()

0

4 - Graficos:

4.1 - grafico para analise de outliers

In [7]:
f1=px.box(df,x='Sales',title='Boxplot de vendas por categoria',template='plotly_dark')
f2=px.box(df,x='Profit',title='Boxplot de lucro por categoria',template='plotly_dark')
f3=px.box(df,x='Quantity',title='Boxplot de quantidade por categoria',template='plotly_dark')
f4=px.box(df,x='Discount',title='Boxplot de desconto por categoria',template='plotly_dark')

fig=make_subplots(rows=2,cols=2,subplot_titles=('Boxplot de vendas por categoria',
                                                'Boxplot de lucro por categoria',
                                                'Boxplot de quantidade por categoria',
                                                'Boxplot de desconto por categoria'))
fig.add_trace(f1['data'][0],row=1,col=1)
fig.add_trace(f2['data'][0],row=1,col=2)
fig.add_trace(f3['data'][0],row=2,col=1)
fig.add_trace(f4['data'][0],row=2,col=2)

fig.update_layout(title_x=0.5,template='plotly_dark',height=600)
fig.show()

4.2 - grafico de analise dos 10 produtos mais vendidos:

In [8]:
Product_Sales = df.groupby('Product Name',as_index=False)['Sales'].sum().round().sort_values('Sales',ascending=False).head(10)
fug=px.bar(Product_Sales,
           y='Sales',x='Product Name'
           ,title='Top 10 produtos mais vendidos',
            template='plotly_dark',
           labels={'Sales':'total de vendas','Product Name':'nome do produto'})
fug.update_layout(title_x=0.5)
fug.show()

4.3 - grafico de vendas por dia, mês e ano

In [9]:
date_by_sales = df[['Order Date','Sales']]
date_by_sales = date_by_sales.set_index('Order Date')
date_by_sales_by_day = date_by_sales['Sales'].resample('D').sum()
date_by_sales_by_month = date_by_sales['Sales'].resample('ME').sum()
date_by_sales_by_year = date_by_sales['Sales'].resample('YE').sum()

f1=px.line(date_by_sales_by_day,
        title='vendas por dia',
        labels={'value':'vendas diarias','Order Date':'Date'},
        template='plotly_dark')

f2=px.line(date_by_sales_by_month,
        title='vendas por mes',
        labels={'value':'vendas mensais','Order Date':'Date'},
        template='plotly_dark')
f3 =px.line(date_by_sales_by_year,
        title='vendas por ano',
        labels={'value':'vendas anuais','Order Date':'Date'},
        template='plotly_dark')
fig = make_subplots(rows=3, cols=1,shared_xaxes=True,subplot_titles=('vendas por dia','vendas por mês','vendas por ano'))
fig.add_trace(f1['data'][0],row=1,col=1)
fig.add_trace(f2['data'][0],row=2,col=1)
fig.add_trace(f3['data'][0],row=3,col=1)
fig.update_layout(title_x=0.5,template='plotly_dark',height=600, showlegend=False)
fig.show()

4.4 - grafico de vendas por categorias

In [10]:
category_by_sales_profit = df.groupby('Category',as_index=False)[['Sales','Profit']].sum().round().sort_values('Sales',ascending=False)
f1=px.pie(category_by_sales_profit,
          values='Sales',names='Category',
          template='plotly_dark',
          labels={'Sales':'Total Sales','Category':'Category'}
          ,hole=0.5)
f2=px.pie(category_by_sales_profit,
          values='Profit',names='Category',
          template='plotly_dark',
          labels={'Profit':'Total Profit','Category':'Category'},
          hole=0.5)


fig=make_subplots(rows=1,cols=2,subplot_titles=('total de vendas por categoria','total de lucro por categoria'),specs=[[{'type':'domain'},{'type':'domain'}]])
fig.add_trace(f1['data'][0],row=1,col=1)
fig.add_trace(f2['data'][0],row=1,col=2)
fig.update_layout(title_x=0.5,template='plotly_dark')
fig.show()

4.5 - grafico das melhores regiões de vendas baseando nos estados e nas cidades:

In [11]:
City_Sales = df.groupby('City',as_index=False)['Sales'].sum().round().sort_values('Sales',ascending=False).head(10)
State_Sales = df.groupby('State',as_index=False)['Sales'].sum().round().sort_values('Sales',ascending=False).head(10)
Region_Sales = df.groupby('Region',as_index=False)['Sales'].sum().round().sort_values('Sales',ascending=False).head(10)

f1=px.bar(City_Sales,
          y='Sales',x='City',
          template='plotly_dark',
          labels={'Sales':'Total Sales','City':'City'})
f2=px.bar(State_Sales,
          y='Sales',x='State',
          template='plotly_dark',
          labels={'Sales':'Total Sales','State':'State'})
f3=px.bar(Region_Sales,
          y='Sales',x='Region',
          template='plotly_dark',
          labels={'Sales':'Total Sales','Region':'Region'})

fig=make_subplots(rows=2,cols=2,subplot_titles=('total de vendas por cidade',
                                                'total de vendas por estado',
                                                'total de vendas por região'))
fig.add_trace(f1['data'][0],row=1,col=1)
fig.add_trace(f2['data'][0],row=1,col=2)
fig.add_trace(f3['data'][0],row=2,col=1)

fig.update_layout(title_x=0.5,template='plotly_dark',height=700, showlegend=False)
fig.show()

In [12]:
df["Profit Margin"] = (df["Profit"] / df["Sales"]) * 100
avg_profit_margin = df.groupby("Category")["Profit Margin"].mean().reset_index()

for i in avg_profit_margin["Category"]:
    print("a margem de lucro médio de ",i," é",avg_profit_margin[avg_profit_margin["Category"] == i]["Profit Margin"].values[0].round(2),"%")

Sub_Category_Sales = df.groupby('Sub-Category',as_index=False)['Sales'].sum().round().sort_values('Sales',ascending=False).head(10)
fig=px.bar(Sub_Category_Sales,
           y='Sales',x='Sub-Category',
           template='plotly_dark',
           labels={'Sales':'total de vendas','Sub-Category':'Sub Categoria'})    
fig.update_layout(title='top 10 de vendas de sub-categorias',title_x=0.5)
fig.show()

a margem de lucro médio de  Furniture  é 3.88 %
a margem de lucro médio de  Office Supplies  é 13.8 %
a margem de lucro médio de  Technology  é 15.61 %
